In [1]:
!pip install opencv-python

In [2]:
#chia ảnh thành các khối 8x8 pixel
import numpy as np
import cv2

#đọc ảnh ở chế độ xám ra thành các mảng 2 chiều
image = cv2.imread('/Users/nguyenquochung/Desktop/IMG_3106.jpeg', cv2.IMREAD_GRAYSCALE) 

#lưu kích thước của ảnh
height,width = image.shape
print("ảnh này có kích thước {0}x{1} pixel".format(height,width))

#lưu các khối 8x8 pixel của ảnh vào mảng 2 chiều 
block = np.array([image[row*8:row*8+8,colum*8:colum*8+8] for (row,colum) in np.ndindex(height//8,width//8)])
print("ảnh chia được {0} khối 8x8 pixel".format(len(block)))

ảnh này có kích thước 4032x3024 pixel
ảnh chia được 190512 khối 8x8 pixel


Công thức biến đổi cosine rời rạc (DCT) thuận:
$ X(k_1,k_2)= \frac{\varepsilon_{k1}\varepsilon_{k2}}{4}\sum\limits_{n1=0}^{7}\sum\limits_{n2=0}^{7}x(n_1,n_2)cos\frac{(2_{n1}+1)k_1\pi}{16}cos\frac{(2_{n2}+1)k_2\pi}{16} $


$\varepsilon_{k}=$
\begin{cases}
\frac{1}{\sqrt{2}}\hspace{1em}  khi \hspace{1em} k=0 \\
0  \hspace{1em}khi\hspace{1em} 1<k<8
\end{cases}

In [3]:
#biến đổi DCT thuận các điểm ảnh 
from scipy.fftpack import dct
def dct_transform(block):
    return dct(dct(block.T, norm='ortho').T, norm='ortho')
dct_block = np.array([dct_transform(block[i]) for i in range(len(block))])
print(dct_block[0])

[[ 2.64125000e+02  1.57720092e+01 -1.60866441e+01  2.99141935e+00
  -1.25000000e-01  5.94189272e-01  3.36539198e-02  5.04554445e-01]
 [ 1.77282699e+01 -7.83102045e+00  1.40560874e+01 -9.31711649e+00
  -3.74130145e+00  4.88755749e+00 -6.74370390e-02  5.87864518e-01]
 [-1.19127716e+01  1.03526252e+01  3.38388348e-01 -3.39028889e-01
  -5.54808490e+00 -3.01081220e-02 -1.09834957e-01 -4.83542650e-02]
 [ 3.35153252e+00 -1.74333448e-01 -1.05725665e-02  1.56523852e-01
  -1.04225950e-01  2.14172314e-01  5.29598444e-01 -2.89665748e-02]
 [ 4.12500000e+00 -1.46984450e-01 -5.85631970e-01  3.44874224e-02
  -1.25000000e-01  1.73379981e-01 -4.33918421e-01  9.82118698e-02]
 [-5.17421418e+00  1.82772497e-01 -3.99911842e-01  2.87395619e-01
  -1.94184158e-01  1.59340139e-01 -3.24943459e-01  2.04346209e-01]
 [-3.42230394e-01 -2.67212550e-01  6.40165043e-01  5.68199803e-02
  -1.99141572e-03 -7.13142906e-02  1.61611652e-01  1.58229639e-02]
 [ 1.25233779e-01  1.61087823e-01 -4.77329860e-01 -2.66237150e-01
  -

lượng tử hoá các khối DCT theo bảng lượng tử chuẩn (Standard Quantization Table)

In [4]:
#bảng lượng tử tiêu chuẩn
quantization_table = np.array([
    [16,  11,  10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 59, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
])
#các khối sau khi lượng tử hoá
quantized_block=np.array([np.round(dct_block[i]/quantization_table).astype(int) for i in range(len(block))])
print(quantized_block[99])

[[19  1  1  0  0  0  0  0]
 [-2 -1  1  0  0  0  0  0]
 [ 1 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]


xây dựng bảng tần số . Bảng tần số là số lần xuất hiện của các giá trị đã qua lượng tử

In [5]:
from collections import defaultdict
frequency_table = defaultdict(int) # tạo dict rỗng
for block in quantized_block:
    for row in block:
        for value in row:
            frequency_table[value] += 1 # bảng tần số 
for value,f in frequency_table.items():
    print("giá trị : {0} , tần số : {1}".format(value,f))

giá trị : 17 , tần số : 4223
giá trị : 1 , tần số : 208179
giá trị : -2 , tần số : 26450
giá trị : 0 , tần số : 11490394
giá trị : -1 , tần số : 206812
giá trị : 16 , tần số : 5139
giá trị : 15 , tần số : 5757
giá trị : 13 , tần số : 9194
giá trị : 14 , tần số : 7282
giá trị : 3 , tần số : 9370
giá trị : 2 , tần số : 26911
giá trị : 18 , tần số : 3888
giá trị : 22 , tần số : 3013
giá trị : -3 , tần số : 9217
giá trị : 4 , tần số : 4252
giá trị : -4 , tần số : 4095
giá trị : 39 , tần số : 1522
giá trị : -7 , tần số : 1044
giá trị : 12 , tần số : 9736
giá trị : 36 , tần số : 1543
giá trị : 6 , tần số : 1526
giá trị : 5 , tần số : 2532
giá trị : 19 , tần số : 3722
giá trị : 23 , tần số : 2861
giá trị : 24 , tần số : 2899
giá trị : 21 , tần số : 3228
giá trị : 28 , tần số : 2223
giá trị : 20 , tần số : 3667
giá trị : 29 , tần số : 1936
giá trị : 25 , tần số : 2678
giá trị : 27 , tần số : 2396
giá trị : 35 , tần số : 1463
giá trị : 37 , tần số : 1499
giá trị : 40 , tần số : 1399
giá trị : 4

tạo cây huffman từ bảng tần số

In [6]:
import heapq
class Node :
    def __init__ (self,frequency,value):
        self.frequency = frequency
        self.value = value
        self.left = None
        self.right = None
def build_huffman_tree(frequency_table):
    heap = []
    for value,frequency in frequency_table.items():
        node = Node(frequency,value)
        heapq.heappush(heap,(node.frequency,id(node),node))
    while(len(heap)) > 1:
        fre1,_,node1 = heapq.heappop(heap)
        fre2,_,node2 = heapq.heappop(heap)
        
        newNode = Node(fre1+fre2,None)
        newNode.left = node1
        newNode.right = node2
        
        heapq.heappush(heap,(newNode.frequency,id(newNode),newNode))
    #trả về cây gốc
    _,_,root = heapq.heappop(heap)
    return root

In [7]:
huffman_tree = build_huffman_tree(frequency_table)

tạo mã code huffman

In [8]:
def generate_huffman_codes(root, current_code, huffman_codes):
    if root is None :
        return
    if root.value is not None :
        huffman_codes[root.value] = current_code
        return
    generate_huffman_codes(root.left,current_code + "0",huffman_codes)
    generate_huffman_codes(root.right,current_code + "1",huffman_codes)

In [9]:
huffman_codes = {}
generate_huffman_codes(huffman_tree,"",huffman_codes)
x = 0
for value,code in huffman_codes.items():
    print("value:{0} , code:{1}".format(value,code))
    x += ((abs(value)*len(code))/8)

value:62 , code:0000000000
value:102 , code:0000000001
value:8 , code:000000001
value:106 , code:000000010
value:115 , code:0000000110
value:63 , code:0000000111
value:21 , code:00000010
value:51 , code:0000001100
value:52 , code:0000001101
value:49 , code:0000001110
value:55 , code:0000001111
value:73 , code:0000010000
value:54 , code:0000010001
value:64 , code:0000010010
value:50 , code:0000010011
value:34 , code:000001010
value:107 , code:000001011
value:58 , code:0000011000
value:53 , code:0000011001
value:86 , code:000001101
value:119 , code:00000111000
value:-10 , code:00000111001
value:45 , code:0000011101
value:56 , code:0000011110
value:89 , code:0000011111
value:2 , code:00001
value:14 , code:0001000
value:30 , code:000100100
value:114 , code:0001001010
value:47 , code:0001001011
value:20 , code:00010011
value:33 , code:000101000
value:46 , code:0001010010
value:94 , code:00010100110
value:-16 , code:0001010011100
value:-39 , code:0001010011101000
value:-36 , code:00010100111

In [10]:
from PIL import Image

# Đường dẫn đến tệp ảnh
image_path = "/Users/nguyenquochung/Desktop/IMG_3106.jpeg"

# Đọc ảnh
image = Image.open(image_path)

# Chuyển đổi ảnh thành chế độ "L" (đen trắng)
image = image.convert("L")

# Lấy số byte từ ảnh
byte_array = image.tobytes()

# In số lượng byte
print('số byte trước khi nén : ',len(byte_array))
print('số byte sau khi nén : ',round(x))

số byte trước khi nén :  12192768
số byte sau khi nén :  14355
